# Generate a Network Model Grid on an OpenTopography DEM for the Puyallup River
<hr>
<small>For more Landlab tutorials, click here: <a href="https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html">https://landlab.readthedocs.io/en/latest/user_guide/tutorials.html</a></small>

<hr>

This notebook demonstrates how to create a NetworkModelGrid from a DEM hosted by OpenTopography. In this tutorial we will:
* Download a DEM from OpenTopography
* Reproject the DEM into a meter-based coordinate system (UTM-13)
* Clip the DEM to the largest watershed
* Create a NetworkModelGrid on the river system in this watershed using three different 'network_grid_from_raster' options

<hr>

*Note: Updated 5/21/2024 to use rioxarray*

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import rioxarray as rxr

import pandas as pd

from landlab import imshow_grid
from landlab.components import FlowDirectorSteepest, NetworkSedimentTransporter
from landlab.data_record import DataRecord
from landlab.grid.network import NetworkModelGrid
from landlab.plot import graph
from landlab.io import read_shapefile
from landlab.plot import plot_network_and_parcels

from bmi_topography import Topography
from rasterio.enums import Resampling

import shapefile

from landlab import RasterModelGrid

%matplotlib inline

## 1. Download DEM from OpenTopography using Topography utility

In [ ]:
width = 0.05
dem = Topography(
    north=47.3358357629999986,
    south=46.7825014000000010,
    east=-121.3748169240000010,
    west=-122.4685226619999980 ,
    output_format="GTiff",
    dem_type="SRTMGL3",
    api_key='0b1844d3d39d5e6fb414e7d6f0ff6e3d'
)
dem.fetch()
_ = dem.load()

In [ ]:
dst_crs="EPSG:32610"   # Destination coordinate reference system: UTM Zone 10 N (Washington)
scale_factor=2
transformed_array=dem.da.rio.reproject(dst_crs).astype('float')

reproj_array = transformed_array.rio.reproject(
    transformed_array.rio.crs,
    shape=(int(transformed_array.sizes['y']/scale_factor), int(transformed_array.sizes['x']/scale_factor)),
    resampling=Resampling.bilinear,
)

## 2. Clip Array Using Watershed Boundary Shapefile

In [ ]:
datadir = pathlib.Path("puyallup_data")
shp_file = datadir / "puyallup_boundary.shp"

boundary = shapefile.Reader(shp_file)

In [ ]:
clipped_array = reproj_array.rio.clip(boundary.shapes())
clipped_bounds=clipped_array.rio.bounds()

In [ ]:
no_dataval = -9999
mask_array = clipped_array.copy()
mask_array.data = no_dataval*np.ones_like(clipped_array.data)
mask_array = mask_array.rio.clip(boundary.shapes(),invert=True)
masked_clipped_array = mask_array + clipped_array
masked_clipped_array.attrs=reproj_array.attrs # Not sure if I need to do this
print(masked_clipped_array)
masked_clipped_array.rio.set_nodata(no_dataval) # Not sure if I need to do this

## 3. Create Landlab Grid

In [ ]:
target_res=50
grid = RasterModelGrid(
    (masked_clipped_array.sizes['y'], masked_clipped_array.sizes['x']),
    xy_spacing=target_res,
    xy_of_lower_left=(clipped_bounds[0],clipped_bounds[1]),
    xy_axis_name=("X UTM10", "Y UTM10"),
    xy_axis_units="m",
)
z = np.ravel(np.flipud(masked_clipped_array.values[0]))
grid.at_node["topographic__elevation"] = z

In [ ]:
grid.status_at_node[np.isclose(z, masked_clipped_array.rio.nodata)] = grid.BC_NODE_IS_CLOSED
# I think I still need to do this

_Note: I deleted the section in the tutorial where it ran FlowAccumulator, since we figured out the watershed boundary with the shapefile. However, I did try FlowAccumulator, and it did not run._

## 4. Set a watershed boundary condition for your grid

In [ ]:
grid.at_node["topographic__elevation"].data[141230]=-9999

In [ ]:
 grid.set_watershed_boundary_condition(
     'topographic__elevation',
     nodata_value=-9999,
     return_outlet_id=True
 )

In [ ]:
grid.set_watershed_boundary_condition_outlet_id(
    [218844],
    "topographic__elevation",
)

In [ ]:
mca_v=xr.where(masked_clipped_array.values>masked_clipped_array.rio.nodata,masked_clipped_array.values,np.nan).ravel()
print(np.nanmean(mca_v),np.nanmax(mca_v))

## 5. Create a NetworkModelGrid on this topography

In [ ]:
from landlab.grid.create_network import (
    AtMostNodes,
    SpacingAtLeast,
    network_grid_from_raster,
    spacing_from_drainage_area,
)
from landlab.plot.graph import plot_links, plot_nodes

In [ ]:
network_grid = network_grid_from_raster(
    grid,
    minimum_channel_threshold=5000000,  # upstream drainage area to truncate network, in m^2
    include=["drainage_area", "topographic__elevation"],
)

In [ ]:
imshow_grid(
    grid,
    "topographic__elevation",
    plot_name="Basin topography",
    color_for_closed=None,
    colorbar_label="$z$ [m]",
)

# Plot network_grid (river channel) 
plot_links(network_grid, with_id=False, as_arrow=False, linewidth=0.05)
plot_nodes(network_grid, with_id=False, markersize=0.2)

# Plot outlet
coords_outlet=grid.xy_of_node[218844]
plt.plot(coords_outlet[0],coords_outlet[1],'ro')

In [ ]:
def my_plot_links(
    graph, color="b", linestyle="solid", with_id=True, as_arrow=True, linewidth=None, textsize=16
):
    if as_arrow:
        head_width = 0.1
    else:
        head_width = 0.0
    for link, nodes in enumerate(graph.nodes_at_link):
        x, y = graph.x_of_node[nodes[0]], graph.y_of_node[nodes[0]]
        dx, dy = graph.x_of_node[nodes[1]] - x, graph.y_of_node[nodes[1]] - y
        plt.arrow(
            x,
            y,
            dx,
            dy,
            head_width=head_width,
            linewidth=linewidth,
            length_includes_head=True,
            color=color,
            linestyle=linestyle,
        )
        if with_id:
            plt.text(x + dx * 0.5, y + dy * 0.5, link, size=textsize, color=color)

In [ ]:
plt.figure(figsize=(10,8))
spacing = spacing_from_drainage_area(
    grid.at_node["drainage_area"], a=9.68, b=0.32, n_widths=100
)

network_grid = network_grid_from_raster(
    grid,
    reducer=SpacingAtLeast(grid.xy_of_node, spacing),
    minimum_channel_threshold=5000000,
    include=["drainage_area", "topographic__elevation"],
)

imshow_grid(
    grid,
    "topographic__elevation",
    plot_name="Basin topography",
    color_for_closed=None,
    colorbar_label="$z$ [m]",
)

plot_nodes(network_grid, with_id=False, markersize=2)
my_plot_links(network_grid, with_id=True, as_arrow=False, textsize=8)
plt.plot(coords_outlet[0],coords_outlet[1],'ro', markersize=6)
plt.title("Nodes and Links");

**There are problems with the network grid plotted above.** Link numbers are _not_ ordered from upstream to downstream (which I think Network Sediment Transporter assumes is true). For example, link 197 - near the northernmost edge of the watershed - is upstream of 196. I believe this is one of the issues that causes problems with `locate_parcel_xy` in `plot_network_and_parcels` below.

## 6. Create Sediment Parcels and Populate Network Grid

Note: the following characteristics of the network model grid are set in the river channel shapefile in the tutorial. Because we determine the network grid from topography, we need to set them. These values may not be physically reasonable.

In [ ]:
network_grid.at_node["bedrock__elevation"] = network_grid.at_node["topographic__elevation"].copy()

network_grid.at_link["channel_width"] = 85 * np.ones(network_grid.number_of_links) # m 85
network_grid.at_link["flow_depth"] = 4 * np.ones(network_grid.number_of_links) # m
network_grid.at_link["reach_length"] = network_grid.length_of_link

In [ ]:
# element_id is the link on which the parcel begins. 
#assign stream order width and depth?
element_id = np.repeat(np.arange(network_grid.number_of_links), 7500)
element_id = np.expand_dims(element_id, axis=1)

volume = 1*np.ones(np.shape(element_id))  # (m3) adjusted from 1
active_layer = np.ones(np.shape(element_id)) # 1= active, 0 = inactive
density = 2650 * np.ones(np.size(element_id))  # (kg/m3)
abrasion_rate = 0 * np.ones(np.size(element_id)) # (mass loss /m)
distance_traveled= 0 * np.ones((np.size(element_id),1)) #(m)

# Lognormal GSD
medianD = 0.001 # m (medium sand diameter, 0.5cm or 0.0005m) 10^-1, 0.10=(10^-2, 10^0)
mu = np.log(medianD)
sigma = np.log(2) #assume that D84 = sigma*D50
np.random.seed(0)
D = np.random.lognormal(
    mu,
    sigma,
    np.shape(element_id)
)  # (m) the diameter of grains in each parcel

In [ ]:
time_arrival_in_link = np.random.rand(np.size(element_id), 1) 
location_in_link = np.random.rand(np.size(element_id), 1) 

In [ ]:
lithology = ["andesite"] * np.size(element_id) #where we will insert characteristics 
#magnetic suscepeblilty, iron oxide characteristics, etc.

In [ ]:
variables = {
    "abrasion_rate": (["item_id"], abrasion_rate),
    "density": (["item_id"], density),
    "lithology": (["item_id"], lithology),
    "time_arrival_in_link": (["item_id", "time"], time_arrival_in_link),
    "active_layer": (["item_id", "time"], active_layer),
    "location_in_link": (["item_id", "time"], location_in_link),
    "distance_traveled": (["item_id", "time"], distance_traveled),
    "D": (["item_id", "time"], D),
    "volume": (["item_id", "time"], volume)
}

In [ ]:
items = {"grid_element": "link", "element_id": element_id}

parcels = DataRecord(
    network_grid,
    items=items,
    time=[0.0],
    data_vars=variables,
    dummy_elements={"link": [NetworkSedimentTransporter.OUT_OF_NETWORK]},
)

## 7. Run Network Sediment Transporter

In [ ]:
timesteps = 13 # total number of timesteps
dt = 60 * 60 * 24 *30 # length of timestep (month)
#dt = 60
#dt = 60 * 30* 1 # halfnhour time steps

In [ ]:
fd = FlowDirectorSteepest(network_grid, "topographic__elevation")
fd.run_one_step()

In [ ]:
D=pd.DataFrame()
D['active_links']=network_grid.active_links
D['flow_direction']=network_grid.at_link['flow__link_direction']
problem_links = list(D[D['flow_direction']==0]['active_links'])
display(problem_links)
coords=np.array([list(network_grid.xy_of_link[x]) for x in problem_links])

In [ ]:
network_grid.at_link['flow__link_direction']=np.array(list([x if (x != 0) else -1 for x in network_grid.at_link['flow__link_direction']]))

In [ ]:
nst = NetworkSedimentTransporter(    
    network_grid,
    parcels,
    fd,
    bed_porosity=0.3,
    g=9.81,
    fluid_density=1000,
    transport_method="WilcockCrowe", #discuss other transport methods, FLVB
)

In [ ]:
time_range=range(0, (timesteps * dt), dt)
sediment_volume=np.zeros_like(time_range)
item_ids=parcels.dataset['item_id'].values
i=0

for t in range(0, (timesteps * dt), dt):
    nst.run_one_step(dt)
    
    distances=np.array(nst._distance_traveled_cumulative)
    parcels.dataset['distance_traveled']=xr.where(parcels.dataset.time==t,
                                                  distances,parcels.dataset['distance_traveled'])
    #print(parcels.dataset.sel(time=t)['distance_traveled'].values)
    #print(distances)
    

    print("Model time: ", t / (60 * 60 * 24), "days passed")
    sediment_volume[i]=network_grid.at_link['sediment_total_volume'].sum()
    i+=1
    

## 8. Plot Parcels and Network

In [ ]:

plt.plot(np.array(time_range)/(60*60*24),sediment_volume,'k.')
plt.xlabel('Time [days]')
plt.ylabel('Total sediment volume in network')
plt.show()

Added the following for error checking.

I think the issue with plotting has to do with some links in the network model grid where the flow directions are 0 (a case that `locate_parcel_xy` does not handle). These seem to occur near confluences, and may be related to places where the numbering does not increase downstream.

Here is where the problems happen (orange crosses):

In [ ]:

spacing = spacing_from_drainage_area(
    grid.at_node["drainage_area"], a=9.68, b=0.32, n_widths=100
)

imshow_grid(
    grid,
    "topographic__elevation",
    plot_name="Basin topography",
    color_for_closed=None,
    colorbar_label="$z$ [m]",
)

plot_nodes(network_grid, with_id=False, markersize=2)
my_plot_links(network_grid, with_id=True, as_arrow=False, textsize=8)
plt.plot(coords[:,0],coords[:,1],'+',color='orange',markersize=12)
plt.title("Nodes and Links");

For now, let's just fix the problem by setting the flow directions in the problem links to -1.

I had to add the following code in to allow us to track the originating links of each parcel for plotting

In [ ]:
originating_links=np.reshape(np.repeat(parcels.dataset.element_id[:,0].values,np.array(time_range).shape[0]+1),(parcels.dataset.element_id[:,0].values.shape[0],np.array(time_range).shape[0]+1)).T
parcels.dataset['originating_link']=xr.DataArray(originating_links,coords=parcels.dataset.coords)

In [ ]:
timestep_of_interest = 1
originating_link = 7 # This is Emmons Glacier (I think)


# filter the parcels to calculate total volumes of only the parcels that originated in the chosen link
parcelfilter = np.zeros_like(parcels.dataset.element_id, dtype=bool)
parcelfilter[:, timestep_of_interest] = (
    parcels.dataset.element_id[:, 0] == originating_link #sets parcel values to either 1 or 0, if it is from origninating link or not
)

vol_orig_link = parcels.calc_aggregate_value(
    xr.Dataset.sum, "volume", at="link", filter_array=parcelfilter, fill_value=0.0 #sums up volume of each link, determines if each parcel in that link is from original link
)


#fig.suptitle('{} days'.format(timestep_of_interest/8))
plt.show()

with pd.option_context("display.max_rows", 1000):
   display(pd.DataFrame(data=np.array([network_grid.active_links,vol_orig_link]).T,columns=['link','volume']))

In [ ]:
print(network_grid.length_of_link[originating_link])

In [ ]:
print(sediment_volume[originating_link])

# Percent Finer Distribution:
## It is saying that the data values we are plotting have a different number of dimensions, which I do not know how to interpret

In [ ]:
particle_sizes = [np.array(parcels.dataset['D'])]
print(particle_sizes)


In [ ]:
sorted_sizes = sorted(particle_sizes)
print(sorted_sizes)


In [ ]:

#percent_finer = np.arange(1, len(sorted_sizes) + 1) / len(sorted_sizes) * 100



In [ ]:

import numpy as np
import matplotlib.pyplot as plt

# Example data array of grain sizes (could represent any units like microns, etc.)
grain_sizes = sorted_sizes
# Assuming each grain size is one "particle" or one unit of volume
# You can replace this with the actual volume or particle counts if you have them
volumes = [1] * len(grain_sizes)  # All volumes are assumed to be 1 unit for simplicity

# Sort the grain sizes in ascending order (smallest to largest)

sorted_volumes = np.sort(volumes)

# Calculate the cumulative volume (or count) passing through each grain size
cumulative_volume = np.cumsum(sorted_volumes)

# Calculate the percent finer by dividing the cumulative volume by the total volume
total_volume = cumulative_volume[-1]
percent_finer = (cumulative_volume / total_volume) * 100

# Print the percent finer distribution
print("Grain Sizes (m):", sorted_sizes)
print("Percent Finer (%):", percent_finer)




In [ ]:
# Optional: Plot the percent finer distribution
plt.plot(sorted_grain_sizes, percent_finer, marker='o', linestyle='-', color='b')
plt.xlabel('Grain Size (microns)')
plt.ylabel('Percent Finer (%)')
plt.title('Percent Finer Distribution')
plt.grid(True)
plt.show()

In [ ]:
#parcel_filter = np.zeros((parcels.dataset.dims["item_id"]), dtype=bool)
#parcel_filter[::500] = True
import matplotlib.colors as colors
from matplotlib.colors import Normalize
import matplotlib as mpl

cmap = mpl.cm.PuRd_r

parcel_color_norm = mpl.colors.Normalize(-1, 300) # Linear normalization
parcel_color_norm = colors.LogNorm(vmin=0.0001, vmax=0.01)  #ADJUST THIS WHEN SETTING MEDIAN DIAMETER
#originating_link=7


# filter the parcels to calculate total volumes of only the parcels that originated in the chosen link
parcel_filter = np.zeros_like(parcels.dataset.element_id, dtype=bool)
for t in range(0,timesteps):
    parcel_filter[:, t] = (
        parcels.dataset.element_id[:, 0] == originating_link #sets parcel values to either 1 or 0, if it is from origninating link or not
    )

    
pc_opts= {
        "parcel_color_attribute": "D", # a more complex normalization and a parcel filter. 
        "parcel_color_norm": parcel_color_norm,
        "parcel_color_attribute_title":"Diameter [m]",
        "parcel_alpha": 1.0,
        "parcel_size": 40,
        "parcel_time_index":0,
        "parcel_filter": parcel_filter[:,0],
        "parcel_color_cmap" : cmap
    }
fig = plot_network_and_parcels(
    network_grid, parcels,
    **pc_opts,
)
plt.show()

In [ ]:
pc_opts2= {
        "parcel_color_attribute": "D", # a more complex normalization and a parcel filter. 
        "parcel_color_norm": parcel_color_norm,
        "parcel_color_attribute_title":"Diameter [m]",
        "parcel_alpha": 1.0,
        "parcel_size": 30,
        "parcel_time_index":12,
        "parcel_filter": parcel_filter[:,0],
        "parcel_color_cmap" : cmap
    }
fig = plot_network_and_parcels(
    network_grid, parcels, 
    **pc_opts2
    )
plt.show()

The following statement lists out how far each parcel from the originating link has traveled at the timestep of intrest. Note that a few have gone over 20 km.

In [ ]:
print(parcels.dataset["distance_traveled"].where((parcels.dataset.time==timestep_of_interest*dt)&(parcels.dataset.originating_link==originating_link),drop=True).values.flatten().T)

In [ ]:
fig,ax = plt.subplots(2,1)
ax[0].hist(parcels.dataset["distance_traveled"].where((parcels.dataset.time==0),drop=True).values.flatten().T,bins=100)
ax[1].hist(parcels.dataset["distance_traveled"].where((parcels.dataset.time==dt*timestep_of_interest),drop=True).values.flatten().T,bins=100)
plt.show()

In [ ]:
fig,ax = plt.subplots(2,1)
ax[0].hist(parcels.dataset['D'].where((parcels.dataset.time==0),drop=True).values.flatten().T,bins=100)
ax[1].hist(parcels.dataset['D'].where((parcels.dataset.time==dt*timestep_of_interest),drop=True).values.flatten().T,bins=100)
plt.show()

Plot of distance traveled vs. time:

In [ ]:
fig, ax = plt.subplots(figsize=(6, 1), layout='constrained')

cmap = mpl.cm.cool
norm = mpl.colors.Normalize(vmin=5, vmax=10)

fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
             cax=ax, orientation='horizontal', label='Some Units')

In [ ]:
parcel_subset=parcels.dataset.where((parcels.dataset.originating_link==originating_link),drop=True)
parcel_subset_df=parcel_subset.to_dataframe().reset_index()
fig, ax = plt.subplots()

p=ax.scatter(parcel_subset_df['time']/(60*60),parcel_subset_df['distance_traveled'],c=parcel_subset_df['D'])
cb=fig.colorbar(p)


plt.xlabel('Time (hours)')
plt.ylabel('Distance Traveled (m)') 
plt.ylim(0,1000)
cb.ax.set_ylabel('Mean Particle Diameter (m)') 
plt.show()

In [ ]:
fig = plot_network_and_parcels(
    network_grid, parcels, 
    parcel_time_index=0, 
    #parcel_color_attribute="D",
    link_attribute="sediment_total_volume", 
    parcel_size=10, 
    parcel_alpha=0,
)

In [ ]:
parcel_subset=parcels.dataset.where((parcels.dataset.originating_link==originating_link),drop=True)
parcel_subset_df=parcel_subset.to_dataframe().reset_index()
fig, ax = plt.subplots()
p=ax.scatter(parcel_subset_df['time']/(60*60),parcel_subset_df['distance_traveled'],c=parcel_subset_df['D'])


cb=fig.colorbar(p)
plt.xlabel('Time (hours)')
plt.ylabel('Distance Traveled (m)') 
plt.ylim(0,7500)
plt.xlim(0,10)
cb.ax.set_ylabel('Mean Particle Diameter (m)') 
plt.show()

# sediment total volume

In [ ]:
plt.loglog(parcels.dataset.D[:,-1],
         nst._distance_traveled_cumulative,
         '.'
        )
plt.ylim(1,10000)
plt.xlabel('Parcel grain size (m)')
plt.ylabel('Cumulative parcel travel distance')

# Note: some of the smallest grain travel distances can exceed the length of the 
# grid by "overshooting" during a single timestep of high transport rate

# parcels moving out of system

In [ ]:
out_of_system=parcels.dataset.where(parcels.dataset.element_id==-2,drop=True).item_id
print(out_of_system)

## Plotting Sediment Volume

In [ ]:
from matplotlib.colors import Normalize
network_norm = Normalize(-1, 13000) # see matplotlib.colors.Normalize (************SEDIMENT VOLUME SCALE)
 
link_color_options = [
    {
        "link_attribute": "sediment_total_volume", 
        "network_norm": network_norm, # and normalize color scheme
        "link_attribute_title": "Total Sediment Volume", # title on link color legend
        "network_cmap": "plasma",
        "parcel_alpha":0, 
        "network_linewidth":3 

    }
]

In [ ]:
for grid, parcels in zip([grid], [parcels]):
    for l_opts in link_color_options:
        fig = plot_network_and_parcels(
        network_grid, parcels, 
        parcel_time_index=0, **l_opts)
    ax=plt.gca()
    #ax.plot(coords[:,0],coords[:,1],'+',color='orange',markersize=12)
plt.show()
#plot_nodes(network_grid, with_id=False, markersize=2)
#my_plot_links(network_grid, with_id=True, as_arrow=False, textsize=8)
#plt.plot(coords[:,0],coords[:,1],'+',color='orange',markersize=12)

In [ ]:
for l_opts in link_color_options:
    fig = plot_network_and_parcels(
        network_grid, parcels, 
        parcel_time_index=9, **l_opts)
    ax=plt.gca()
    #ax.plot(coords[:,0],coords[:,1],'+',color='orange',markersize=12)
plt.show()

## 6. Plotting a subset of the parcels

In some cases, we might want to plot only a subset of the parcels on the network. Below, we plot every 500th parcel in the `DataRecord`. 

In [ ]:
import matplotlib as mpl
fig, ax = plt.subplots(figsize=(6, 1), layout='constrained')

cmap = mpl.cm.PuBu_r
norm = mpl.colors.Normalize(vmin=5, vmax=10)

fig.colorbar(mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
             cax=ax, orientation='horizontal', label='Some Units')

In [ ]:
#parcel_filter = np.zeros((parcels.dataset.dims["item_id"]), dtype=bool)
#parcel_filter[::500] = True
import matplotlib.colors as colors
from matplotlib.colors import Normalize
import matplotlib as mpl

cmap = mpl.cm.BuPu_r

parcel_color_norm = mpl.colors.Normalize(-1, 300) # Linear normalization
parcel_color_norm = colors.LogNorm(vmin=0.0001, vmax=0.01)  #ADJUST THIS WHEN SETTING MEDIAN DIAMETER
originating_link=33


# filter the parcels to calculate total volumes of only the parcels that originated in the chosen link
parcel_filter = np.zeros_like(parcels.dataset.element_id, dtype=bool)
for t in range(0,timesteps):
    parcel_filter[:, t] = (
        parcels.dataset.element_id[:, 0] == originating_link #sets parcel values to either 1 or 0, if it is from origninating link or not
    )

    
pc_opts= {
        "parcel_color_attribute": "D", # a more complex normalization and a parcel filter. 
        "parcel_color_norm": parcel_color_norm,
        "parcel_color_attribute_title":"Diameter [m]",
        "parcel_alpha": 1.0,
        "parcel_size": 40,
        "parcel_time_index":0,
        "parcel_filter": parcel_filter[:,0],
        "parcel_color_cmap" : cmap
    }
fig = plot_network_and_parcels(
    network_grid, parcels,
    **pc_opts,
)
pc_opts2= {
        "parcel_color_attribute": "D", # a more complex normalization and a parcel filter. 
        "parcel_color_norm": parcel_color_norm,
        "parcel_color_attribute_title":"Diameter [m]",
        "parcel_alpha": 1.0,
        "parcel_size": 40,
        "parcel_time_index":48,
        "parcel_filter": parcel_filter[:,0],
        "parcel_color_cmap" : cmap
    }
fig = plot_network_and_parcels(
    network_grid, parcels, 
    **pc_opts2
    )
plt.show()

In [ ]:
#parcel_vol_on_grid = parcels.dataset["volume"].values
#parcel_vol_on_grid[parcels.dataset["element_id"].values==-2]=0
#parcel_filter = np.zeros_like(parcels.dataset.element_id, dtype=bool)
#for t in range(0,timesteps):
   # parcel_filter[:, t] = (
     #   parcels.dataset.element_id[:, 0] == originating_link #sets parcel values to either 1 or 0, if it is from origninating link or not
   # )

#travel_distance=nst._distance_traveled_cumulative,"parcel_filter"== parcel_filter[:,0]
print(travel_distance)

## 4. Options for parcel color

The dictionary below (`parcel_color_options`) outlines 4 examples of link color and line width choices: 

3. Color parcels by an existing parcel attribute, in this case the sediment diameter of the parcel (`parcels1.dataset['D']`)


In [ ]:
import matplotlib.colors as colors

parcel_filter = np.zeros((parcels.dataset.dims["item_id"]), dtype=bool)
parcel_filter[::500] = True



parcel_color_options = [
    {
        "parcel_color_attribute": "D", # existing parcel attribute. 
        "parcel_color_norm": parcel_color_norm,
        "parcel_color_attribute_title":"Diameter [m]",
        "parcel_alpha":1.0,
    },
]

for grid, parcels in zip([grid], [parcels]):
    for pc_opts in parcel_color_options:
        fig = plot_network_and_parcels(
            network_grid, parcels, 
            parcel_time_index=0, **pc_opts)
        parcel_filter
        plt.show()



## 5. Options for parcel size
The dictionary below (`parcel_size_options`) outlines 4 examples of link color and line width choices: 
1. The default output of `plot_network_and_parcels`
2. Set a uniform parcel size and color
3. Size parcels by an existing parcel attribute, in this case the sediment diameter (`parcels1.dataset['D']`), and making the parcel markers entirely opaque. 
4. Normalize parcel size on a logarithmic scale, and change the default maximum and minimum parcel sizes. 

## 7. Select the parcel timestep to be plotted

As a default, `plot_network_and_parcels` plots parcel positions for the last timestep of the model run. However, `NetworkSedimentTransporter` tracks the motion of parcels for all timesteps. We can plot the location of parcels on the link at any timestep using `parcel_time_index`. 

In [ ]:
parcel_time_options = [0,9,20]

for grid, parcels in zip([grid], [parcels]):
    for pt_opts in parcel_time_options:
        fig = plot_network_and_parcels(
            network_grid, parcels, 
            parcel_size = 20,
            parcel_alpha = 0.1,
            parcel_time_index=pt_opts)
        plt.show()